<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Constants</a></span></li><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#NL" data-toc-modified-id="NL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NL</a></span></li><li><span><a href="#MS" data-toc-modified-id="MS-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MS</a></span></li><li><span><a href="#Create-activation-maps-figure" data-toc-modified-id="Create-activation-maps-figure-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create activation maps figure</a></span></li></ul></div>

# Imports and Constants

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from collections import defaultdict
import heapq
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

sys.path.append('../')
from batchers import batcher, dataset_constants
from models.resnet_model import Hyperspectral_Resnet

In [ ]:
CKPTS_DIR = '../ckpts'
LOGS_DIR = '../logs'
NUM_TOP_IMGS = 8

MODEL_PATHS = {
    'incountry_resnet_ms_D': [
        'DHSIncountry/incountryD_18preact_ms_samescaled_b64_fc001_conv001_lr0001',
        'ckpt-40'],
    'incountry_resnet_nl_C': [
        'DHSIncountry/incountryC_18preact_nl_random_b64_fc1.0_conv1.0_lr0001',
        'ckpt-145']
}

BATCH_SIZE = 128
LABEL_NAME = 'wealthpooled'
GPU = 0
GPU_USAGE = 0.9
IS_TRAINING = False
MODEL_PARAMS = {
    'fc_reg': 5e-3,
    'conv_reg': 5e-3,
    'num_layers': 18,
    'num_outputs': 1
}

TENSOR_NAMES = {
    'scale1_img': 'resnet/scale1/scale1_img:0',  # batch_size, 112, 112,  64
    'scale2_img': 'resnet/scale2/scale2_img:0',  # batch_size,  56,  56,  64
    'scale3_img': 'resnet/scale3/scale3_img:0',  # batch_size,  28,  28, 128
    'scale4_img': 'resnet/scale4/scale4_img:0',  # batch_size,  14,  14, 256
}

# Helper Functions

In [ ]:
def get_dhs_test_batcher(ls_bands, nl_band):
    '''
    Args
    - ls_bands: one of ['rgb', 'ms', None]
    - nl_band: one of ['merge', 'split', None]
    - fold: str, one of ['A', 'B', 'C', 'D', 'E']

    Returns
    - batcher: Batcher
    '''
    all_tfrecord_paths = np.asarray(batcher.get_tfrecord_paths('2009-17', 'all'))
    assert len(all_tfrecord_paths) == dataset_constants.SIZES['2009-17']['all']

    b = batcher.Batcher(
        tfrecord_files=all_tfrecord_paths,
        dataset='2009-17',
        batch_size=BATCH_SIZE,
        label_name=LABEL_NAME,
        num_threads=4,
        epochs=1,
        ls_bands=ls_bands,
        nl_band=nl_band,
        shuffle=False,
        augment=False,
        normalize=True,
        cache=False)
    return b

In [ ]:
def add_to_heap(h, k, value, data):
    '''
    Heap h keeps track of the max k elements

    We will actually use a min-heap for this task. That way, when a new element
    comes in, we compare it to the smallest node in the heap, h[0]. If the new
    value is greater than h[0], we pop h[0] and add the new element in.

    Args
    - h: list, either empty [] or already heapified
    - k: desired capacity of the heap
    - value: value to compare with
    - data: data to store with the value
    ''' 
    if len(h) < k:
        heapq.heappush(h, (value, data))
    else:
        heapq.heappushpop(h, (value, data))

In [ ]:
def run_batches(sess, tensors_dict_ops, tensor_name, max_nbatches=None, verbose=False):
    '''Runs the ops in tensors_dict_ops for a fixed number of batches or until
    reaching a tf.errors.OutOfRangeError, concatenating the runs.

    Note: assumes that the dataset iterator doesn't need initialization, or is
        already initialized.

    Args
    - sess: tf.Session
    - tensors_dict_ops: dict, str => tf.Tensor, shape [batch_size] or [batch_size, D]
    - max_nbatches: int, maximum number of batches to run the ops for,
        set to None to run until reaching a tf.errors.OutOfRangeError

    Returns
    - top_images_avg: dict, maps filter number (int) into list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year, loc, activation map)
    '''
    top_images_avg = defaultdict(list)

    curr_batch = 0
    start_time = time.time()
    try:
        while True:
            all_tensors = sess.run(tensors_dict_ops)

            imgs = all_tensors['images']  # N, H, W, C
            years = all_tensors['years']
            locs = all_tensors['locs']
            actmaps = all_tensors[tensor_name]

            batch_size, _, _, num_filters = actmaps.shape

            actmaps_means = np.mean(actmaps, axis=(1, 2), dtype=np.float64)  # shape [batch_size, num_filters]

            for i in range(batch_size):
                img = imgs[i]
                year = years[i]
                loc = tuple(locs[i])
                actmap = actmaps[i]  # shape [H, W, num_filters]
                actmap_means = actmaps_means[i]  # shape [num_filters]

                for f in range(num_filters):
                    value = (actmap_means[f], -(i + curr_batch*BATCH_SIZE))
                    data = (img, year, loc, actmap[:, :, f])
                    add_to_heap(h=top_images_avg[f], k=NUM_TOP_IMGS, value=value, data=data)

            curr_batch += 1
            if verbose:
                speed = curr_batch / (time.time() - start_time)
                print(f'\rRan {curr_batch} batches ({speed:.3f} batch/s)', end='')
            if (max_nbatches is not None) and (curr_batch >= max_nbatches):
                break
            if curr_batch >= 15:
                break

    except tf.errors.OutOfRangeError:
        pass

    print()  # print a newline, since the previous print()'s don't print newlines
    return top_images_avg

In [ ]:
def get_max_act_images(ckpt_path: str, fold: str, ls_bands: str,
                       nl_band: str, tensor_name: str):
    '''
    Args
    - ckpt_path: str
    - fold: str, one of ['A', 'B', 'C', 'D', 'E']
    - ls_bands: str or None
    - nl_band: str or None
    - tensor_name: str, key of TENSOR_NAMES

    Returns
    - top_images_avg: dict, maps filter number (int) into list of (value, data) tuples
        value = (mean filter activation for a particular image, negative image index)
        data = (img, year, loc, activation map)
    '''
    tf.reset_default_graph()

    print('=== Running model ===')
    print('- ckpt:', ckpt_path)
    print('- fold:', fold)
    print('- ls_bands, nl_band:', ls_bands, nl_band)

    init_iter, batch_op = get_dhs_test_batcher(ls_bands, nl_band).get_batch()

    print('Building model...')
    model = Hyperspectral_Resnet(batch_op['images'], is_training=IS_TRAINING, **MODEL_PARAMS)

    graph = tf.get_default_graph()
    tensors_dict_ops = {
        'images': batch_op['images'],
        'years': batch_op['years'],
        'locs': batch_op['locs'],
        tensor_name: graph.get_tensor_by_name(TENSOR_NAMES[tensor_name])
    }

    print('Creating session...')
    os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)
    config_proto = tf.ConfigProto()
    config_proto.gpu_options.per_process_gpu_memory_fraction = GPU_USAGE

    with tf.Session(config=config_proto) as sess:
        sess.run(init_iter)

        # clear the model weights, then load saved checkpoint
        print('Loading saved ckpt...')
        saver = tf.train.Saver(var_list=None)
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
        saver.restore(sess, ckpt_path)

        # run the saved model
        top_images_avg = run_batches(sess, tensors_dict_ops, tensor_name, verbose=True)
    return top_images_avg

In [ ]:
def plot_activations(imgs, actmaps, locs=None, years=None, title=None, size=4, nl=False, savedir=None):
    '''
    Args
    - imgs: list of np.array, length N, each np.array has shape [H, W] or [H, W, 3]
        - if [H, W, 3], then band order is assumed to be R, G, B
    - actmaps: list of np.array, length N, each np.array has shape [actH, actW]
    - locs: list of (lat, lon) tuples
    - years: list of int
    - title: str, figure title
    - size: int, size of each img, in inches
    - nl: bool, when plotting nightlights images
    - savedir: str, path to directory to save imgs and activation maps
    '''
    nimgs = len(imgs)
    assert len(actmaps) == nimgs

    # make copy, so we don't modify the originals
    np_imgs = np.array(imgs)
    np_actmaps = np.abs(np.array(actmaps))  # activation maps aren't always after a ReLU

    # sort images by mean activation in descending order
    sorted_index = np.argsort(np.mean(np_actmaps, axis=(1, 2)))[::-1]
    np_imgs = np_imgs[sorted_index]
    np_actmaps = np_actmaps[sorted_index]

    if locs is not None:
        locs = np.array(locs)  # make copy
        locs = locs[sorted_index]
    if years is not None:
        years = np.array(years)  # make copy
        years = years[sorted_index]

    max_act = np.percentile(np_actmaps, q=99)

    fig, axs = plt.subplots(2, nimgs, figsize=[size*nimgs, size*2])

    for i in range(nimgs):
        img = np_imgs[i]
        actmap = np_actmaps[i]

        # center images towards mean-0
        img = img / 6
        mean = np.mean(img)
        img -= np.sign(mean) * min(0.9*abs(mean), abs(mean)**1.4)
        new_mean = np.mean(img)

        # scale images towards std-dev 1/6
        std = np.std(img)
        img = (img - new_mean) * (0.16 / std)**0.7 + new_mean
        print('Mean_0:', mean, 'Mean_new:', new_mean, 'Std_0:', std, 'Std_new:', np.std(img))

        img = np.clip(img + 0.5, a_min=0, a_max=1)

        if nl:
            mean = np.mean(actmap)
            std = np.std(actmap)
            actmap_max = min(np.max(actmap), mean + 6 * std)
            actmap_min = max(np.min(actmap), mean - 6 * std)
            actmap = (actmap - actmap_min) / actmap_max
            actmap = np.clip(actmap, a_min=0, a_max=1)
        else:
            actmap = np.clip(actmap, a_min=0, a_max=max_act) / max_act
    
        # origin='lower' to match lat/lon direction
        axs[0, i].imshow(img, origin='lower', vmin=0, vmax=1)
        axs[1, i].imshow(actmap, origin='lower', vmin=0, vmax=1,
                         interpolation='none', cmap='gray')

        axs[0, i].axis('off')
        axs[1, i].axis('off')

        ax_title = []
        if locs is not None:
            lat, lon = locs[i]
            ax_title.append(f'loc: ({lat:.4f}, {lon:.4f})')
        if years is not None:
            year = years[i]
            ax_title.append(f'year: {year}')
        if len(ax_title) > 0:
            ax_title = ' '.join(ax_title)
            axs[0, i].set_title(ax_title)

        if savedir is not None:
            os.makedirs(savedir, exist_ok=True)
            img_filename = os.path.join(savedir, f'img_{i}.png')
            # plt.imsave(img_filename, img, vmin=0, vmax=1, format='png', origin='lower')
            PIL.Image.fromarray((img * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(img_filename, optimize=True)

            actmap_filename = os.path.join(savedir, f'actmap_{i}.png')
            # plt.imsave(actmap_filename, actmap, vmin=0, vmax=1, format='png', origin='lower', cmap='gray')
            PIL.Image.fromarray((actmap * 255).astype(np.uint8)).transpose(PIL.Image.FLIP_TOP_BOTTOM).save(actmap_filename, optimize=True)

    if title is not None:
        fig.suptitle(title, y=1.03)
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout()
    plt.show()

# NL

In [ ]:
actmaps_pkl_path = 'dhs_incountryC_actmaps_nl.pkl'
if not os.path.exists(actmaps_pkl_path):
    ckpt_path = os.path.join(CKPTS_DIR, *MODEL_PATHS['incountry_resnet_nl_C'])
    fold = 'C'
    ls_bands = None
    nl_band = 'split'
    tensor_name = 'scale2_img'

    top_images_avg_nl = get_max_act_images(ckpt_path, fold, ls_bands, nl_band, tensor_name)
    with open(actmaps_pkl_path, 'wb') as f:
        pickle.dump(top_images_avg_nl, f)

In [ ]:
with open(actmaps_pkl_path, 'rb') as f:
    top_images_avg_nl = pickle.load(f)

In [ ]:
for f in range(len(top_images_avg_nl)):
    imgs = []
    locs = []
    actmaps = []

    for value, data in top_images_avg_nl[f]:
        img, year, loc, actmap = data
        C = 0 if year < 2012 else 1
        img = img[:, :, C]
        imgs.append(img)
        locs.append(loc)
        actmaps.append(actmap)

    title = f'Filter {f}'
    plot_activations(imgs, actmaps, locs=locs, title=title, size=2, nl=True)

# MS

In [ ]:
actmaps_pkl_path = 'dhs_incountryD_actmaps_ms.pkl'
if not os.path.exists(actmaps_pkl_path):
    ckpt_path = os.path.join(CKPTS_DIR, *MODEL_PATHS['incountry_resnet_ms_D'])
    fold = 'D'
    ls_bands = 'ms'
    nl_band = None
    tensor_name = 'scale3_img'

    top_images_avg_ms = get_max_act_images(ckpt_path, fold, ls_bands, nl_band, tensor_name)
    with open(actmaps_pkl_path, 'wb') as f:
        pickle.dump(top_images_avg_ms, f)

In [ ]:
with open(actmaps_pkl_path, 'rb') as f:
    top_images_avg_ms = pickle.load(f)

In [ ]:
# for f in sorted(top_images_avg_ms.keys()):
#     imgs = []
#     years = []
#     locs = []
#     actmaps = []

#     for value, data in top_images_avg_ms[f]:
#         img, year, loc, actmap = data
#         img = img[:, :, [2, 1, 0]]  # convert from BGR to RGB
#         imgs.append(img)
#         years.append(year)
#         locs.append(loc)
#         actmaps.append(actmap)

#     title = f'Filter {f}'
#     plot_activations(imgs, actmaps, locs=locs, years=years, title=title)

In [ ]:
ms_filters_to_label = {
    2: 'urban',
    10: 'water',
    20: 'orange',
    45: 'water',
    59: 'senegal_river',
    64: 'farmland',
    79: 'canyons',
    82: 'greenery',
    102: 'greenery',
    105: 'orange'
}

for f in range(len(top_images_avg_ms)):
    if f not in ms_filters_to_label:
        continue

    imgs = []
    years = []
    locs = []
    actmaps = []

    for value, data in top_images_avg_ms[f]:
        img, year, loc, actmap = data
        img = img[:, :, [2, 1, 0]]  # convert from BGR to RGB
        imgs.append(img)
        years.append(year)
        locs.append(loc)
        actmaps.append(actmap)

    title = f'Filter {f:d}'
    savedir = None
    if f in ms_filters_to_label.keys():
        savedir = os.path.join(LOGS_DIR, MODEL_PATHS['incountry_resnet_ms_D'][0], 'actmaps', str(f))
    plot_activations(imgs, actmaps, locs=locs, title=title, savedir=savedir)

# Create activation maps figure

In [ ]:
def images_grid(imgs, size, resize=PIL.Image.NEAREST, spacing=0, color=0):
    '''
    Args
    - imgs: list of list of PIL.Image, 1 sublist for each row of images
    - size: int, width/height in pixels to reshape images to
    - resize: PIL filter
    - spacing: int, number of pixels between adjacent images
    - color: int, spacing color, 0 for black, 255 for white

    Returns: np.array, grid of images
    '''
    nrows = len(imgs)
    ncols = len(imgs[0])

    gridH = nrows * size + (nrows - 1) * spacing
    gridW = ncols * size + (ncols - 1) * spacing
    grid = np.ones([gridH, gridW, 3], dtype=np.uint8) * color

    for r in range(nrows):
        for c in range(ncols):
            img = imgs[r][c]
            if img.size != (size, size):
                img = img.resize((size, size), resize)
            i = r * (size + spacing)
            j = c * (size + spacing)
            grid[i:i+size, j:j+size, :] = np.asarray(img).reshape(size, size, -1)
    return grid

In [ ]:
for f in [2, 10, 64]:
    savedir = os.path.join(LOGS_DIR, MODEL_PATHS['incountry_resnet_ms_D'][0], 'actmaps', str(f))
    top_row, bot_row = [], []
    for i in range(NUM_TOP_IMGS):
        top_row.append(PIL.Image.open(os.path.join(savedir, f'img_{i}.png')))
        bot_row.append(PIL.Image.open(os.path.join(savedir, f'actmap_{i}.png')))
    imgs = [top_row, bot_row]
    grid = images_grid(imgs, size=224, spacing=5, color=255)
    savepath = os.path.join(savedir, 'grid.png')
    PIL.Image.fromarray(grid).save(savepath, optimize=True)